<h1> Exercício 1 -  Mineração de dados</h1>
<h4> by: Luan Carlos Klein </h4>

In [1]:
#Instalação das bibliotecas necessárias
#!pip install TwitterAPI
#!pip install simplejson
#!pip install requests
#!pip install selenium

<h2> Questão 1 </h2>
Faça uma adaptação no coletor de tweets para coletar qualquer tipo de tweet compartilhado no
Brasil. Dica: veja a documentação da biblioteca TwitterApi. Faça uma coleta por alguns minutos,
salvando o resultado em um arquivo ou em um banco de dados.

In [2]:
import time
from TwitterAPI import TwitterAPI
import json
import pandas as pd

## Conectando a API do twitter
twitter_api = TwitterAPI(consumer_key='XXXXXX',
              consumer_secret='XXXXXX',
              access_token_key='XXXXXX',
              access_token_secret='XXXXXX')
## Para pegar a localização: https://boundingbox.klokantech.com/
## Seta como filtro a localização do Brasil
filters = {"locations": '-58.3,-32.5,-34.6,3.3'}
## Passe esse filtro para a request na api
stream = twitter_api.request('statuses/filter', filters).get_iterator()

## Inicia a contagem do tempo
time_start = time.time()
## Define um total de 3 minutos, que será o tempo da coleta dos dados
total_time = 180
## Cria um dataframe vazio
df = pd.DataFrame()
## Percorre para fazer a requisição dos tweets
for item in stream:
    try:
        ## Pega o tempo atual para ver quanto tempo já passou
        time_end = time.time()
        time_spend = time_end - time_start
        ## Se o tempo gasto passou de 2 minutos, então para a coleta dos dados
        if time_spend >= total_time:
            break
        
        ## Verifica se o dataframe está vazio
        ## Se sim, os dados coletados, convertidos em um dataframe serão o novo dataframe
        if len(df.columns) == 0:
            df = pd.DataFrame([item])
        ## Caso já tenha dados no dataframe, apenas adiciona mais dados nele
        else:
            df = pd.concat([df, pd.DataFrame([item])])
            df = df.reset_index(drop=True)
    except Exception as e:
        print(e)
        continue
## Salva os dados coletados em um arquivo .csv
df.to_csv("tweets.csv")
df

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,display_text_range,extended_entities,possibly_sensitive,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink
0,Mon Mar 21 11:07:05 +0000 2022,1505863560248897537,1505863560248897537,Eu to morto feat enterrado,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,pt,1647860825012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mon Mar 21 11:07:05 +0000 2022,1505863563885359106,1505863563885359106,@MichaelCareli @SkillsLevel 🥰 o maior desafio ...,"<a href=""http://twitter.com/download/iphone"" r...",False,1505846312507527174,1505846312507527174,194448167,194448167,...,pt,1647860825879,"[28, 77]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mon Mar 21 11:07:06 +0000 2022,1505863565957287937,1505863565957287937,#FelizAniversarioBolsonaro que Deus abençoe i...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,pt,1647860826373,"[0, 66]","{'media': [{'id': 1505863563499421699, 'id_str...",False,NaN,NaN,NaN,NaN,NaN
3,Mon Mar 21 11:07:08 +0000 2022,1505863574786293761,1505863574786293761,@JornalGGN Empregadoras.,"<a href=""http://twitter.com/download/android"" ...",False,1505863470864031746,1505863470864031746,778721420738125824,778721420738125824,...,es,1647860828478,"[11, 24]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mon Mar 21 11:07:08 +0000 2022,1505863575314878467,1505863575314878467,Bommmm dia!,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,pt,1647860828604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,Mon Mar 21 11:09:55 +0000 2022,1505864276887674881,1505864276887674881,@Leo_am33 @AldoFornazieri @WhiteHouse @10Downi...,"<a href=""http://twitter.com/download/android"" ...",True,1505860074132197382,1505860074132197382,108386128,108386128,...,pt,1647860995872,"[55, 140]",NaN,NaN,{'full_text': '@Leo_am33 @AldoFornazieri @Whit...,NaN,NaN,NaN,NaN
465,Mon Mar 21 11:09:56 +0000 2022,1505864278397444097,1505864278397444097,@Veritosanabria @LuciaRLemos @Jmo2011 @Autocon...,"<a href=""http://twitter.com/download/android"" ...",False,1505861803921739778,1505861803921739778,109825708,109825708,...,es,1647860996232,"[55, 71]","{'media': [{'id': 1505864268465549315, 'id_str...",False,NaN,NaN,NaN,NaN,NaN
466,Mon Mar 21 11:09:57 +0000 2022,1505864281660616706,1505864281660616706,Cómo la sufro cursando,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,es,1647860997010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,Mon Mar 21 11:09:58 +0000 2022,1505864287717371907,1505864287717371907,"Estacio está decadente pqp, acordei atrasada e...","<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,pt,1647860998454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h2> Questão 2 </h2>Com base no dataset gerado na questão 1, faça:
a) Separe todas as hashtags usadas nos Tweets.
b) Descubra a porcentagem de tweets com hashtags.
c) Identifique a hashtag mais popular.

In [3]:
import ast

## Abre o arquivo de tweets salvos anteriormente
df = pd.read_csv("tweets.csv")

######################## (A) - Encontrando as hashtags ##########################
## Pega todos o campo entities (onde estáo as hashtags) de todos os tweets 
entities = df['entities']
## Cria uma lista para armazenar todas as hastags
hashtags = []
## Váriavel que armazena quantos tweets tem hashtagas
qtdTweetsHashtags = 0
## Percorre todas as entidades do tweets
for i in entities:
    ## Transforma o dado em um dicionário do python
    hash_aux = ast.literal_eval(i)
    ## Se houver alguma hashtag, então adiciona mais um no contador de tweets com hashtags
    if len(hash_aux["hashtags"]) > 0:
        qtdTweetsHashtags += 1
        ## Percorre e coloca todas as hashtags encontradas na lista
        for j in hash_aux["hashtags"]:
            hashtags.append(j["text"])
print("Todas hashtags encontradas:")
print(hashtags)
print("Hashtags encontradas (sem duplicação):")
print(list(set(hashtags)))

Todas hashtags encontradas:
['FelizAniversarioBolsonaro', 'jigsawHD', 'puzzletime', 'mayarapinto', 'escolhiesperar', 'euvocedeus', 'DeusNoComando', 'foralais', 'BGManhaRJ', 'foralais', 'ForaEliezer', 'arthurcampeão', 'BBBB22', 'redebbb22', 'academia', 'BBB22', 'divorcio', 'direitocivil', 'direitodasfamilias', 'casamento', 'oab', 'ForaEliezer', 'bbb22', 'CarpeDiem', 'deodoro', 'foralais', 'FicaDG', 'Bomdiadf', 'FelizAniversarioPresidente', 'amazonia', 'metrosp', 'BGManhaRJ', 'BDSP', 'PrimeiroImpacto', 'BalancoGeral', 'BoraBrasil', 'BDSP', 'segundafeira', 'FelizAniversarioBolsonaro', 'maniac_skz', 'StrayKids', 'FelizAniversarioPresidente', 'foralais', 'NP', 'HalaMadridYNadaMas', 'foralais', 'TempoAgora', 'temperatura', 'umidade', 'FelizAniversarioBolsonaro', 'PINTOSAWARD', 'PINTOAWARSDS', 'BUNDAAWARDS', 'foralais', 'diadeflu', 'envolver', 'smile', 'segunda', 'foralais', 'BGManhaRJ', 'foralais', 'BBB22', 'BDSP', 'bomdia', 'Recife', 'Brasil', 'FelizAniversarioBolsonaro', 'ParabensJairBolso

In [4]:
############### (B) - Calculando a porcentagem ################
porcentagemUso = qtdTweetsHashtags / len(df.index) * 100
print(f"Porcentagem de tweets com hashtag: {round(porcentagemUso, 2)}%")

Porcentagem de tweets com hashtag: 8.96%


In [5]:
##################### (C) - Identificando a hashtag mais popular #####################

## Coloca em ordem alfabética as hashtags
hashtags = sorted(hashtags)
## Cria váriaveis para manter a hashtag mais popular até o momento
maxQtdRepetida = 0
hashtagMaisPopular = None

## Cria as váriaveis para manter os dados da hashtag analisada no momento
qtdHashtagAtual = 0
hashtagAtual = None
## Percorre todas as hashtags
for i in hashtags:
    ## Se a hashtag atual for diferente da anterior
    if i != hashtagAtual:
        ## Verifica se ela tem mais ocorrencias que a que mais teve até o momento
        if qtdHashtagAtual >= maxQtdRepetida:
            ## Se sim, considera essa como a mais popular
            maxQtdRepetida = qtdHashtagAtual
            hashtagMaisPopular = hashtagAtual
        ## Reinicia as informações de contagem
        qtdHashtagAtual = 1
        hashtagAtual = i
    else:
        qtdHashtagAtual += 1
    
print(f"Hashtag mais popular é: {hashtagMaisPopular}")
print(f"Ela foi identificada: {maxQtdRepetida} vez(es)")

Hashtag mais popular é: foralais
Ela foi identificada: 8 vez(es)


<h2> Questão 3 </h2>Faça um coletor que capture as reviews da página principal de:
https://www.apontador.com.br/local/sp/campinas/faculdades_e_universidades/88Q2E372/
unicamp.html.


In [6]:
import requests
from bs4 import BeautifulSoup

## Define a url a ser acessar
url = "https://www.apontador.com.br/local/sp/campinas/faculdades_e_universidades/88Q2E372/unicamp.html"

## Retorna o conteudo da pagina
req = requests.get(url)

## Transforma o conteudo da pagina em um objeto BeautifulSoup
soup = BeautifulSoup(req.content,'html.parser')

## Encontra todos os elementos que são div e tem a classe store-comments__item, que é onde estão salvos os reviews. 
dadosBrutos = soup.findAll("div",{"class":"store-comments__item"})


print("Os reviews encontrados são:")
print()
## Percorre todos os reviews e mostra eles na tela
for i in dadosBrutos:
    print (i.text)
    print()

Os reviews encontrados são:

      Renan Cunha De Sousa          publicada em 05/01/2015    Unicamp é umas das mais cobiçadas universidades de Campinas e região, devido a sua imensa qualidade, por ser federal se torna a melhor faculdade da região. Sem contar o seu Campus que é sensacional, como se fosse uma cidade, possui quadras com prédios, até uma linha de ônibus circular que permanece dentro do campus apenas devido o seu tamanho. A universidade conta também com o Hospital das Clínicas, gratuito pelo SUS e administrar o hospital de Sumaré. As provas para ingressar na faculdade são por fases com respostas alternativas e requisitando cerca de 3 redações o número de candidatos por vaga é bem alto principalmente no curso de medicina, o problema fica no pagamento do vestibular que é cerca de 120~140 reais para poder executa-la. A graduação é em tempo integral com isso a faculdade possui centro de pesquisas, onde é possível fazer iniciações cientificas ou mini empresas, para que os alunos

<h2> Questão 4 </h2>
Crie uma aplicação que usa o Selenium para abrir o site https://www.uol.com.br/ e rolar a página
para baixo até o fim do conteúdo.

In [7]:
from selenium import webdriver

##Informa o caminho do webdriver na sua máquina
driver = webdriver.Chrome(executable_path='.\chromedriver')

## Realiza a conexão com o site desejado
driver.get("https://www.uol.com.br/")

## Executa o comando para rolar para o final da página (argumentos da função são X e Y, e termos absolutos)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


C:\Users\luank\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.
